In [60]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os

In [61]:
tf.reset_default_graph()

In [62]:
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [63]:
batch_size = 100
learning_rate = 0.5
training_epochs = 5
logs_path = "tmp/mnist/2"
if not os.path.exists(logs_path):
    os.makedirs(logs_path)

In [64]:
with tf.name_scope('input'):
    # None -> batch size can be any size, 784 -> flattened mnist image
    x = tf.placeholder(tf.float32, shape=[None, 784], name="x-input") 
    # target 10 output classes
    y_ = tf.placeholder(tf.float32, shape=[None, 10], name="y-input")


In [65]:
with tf.name_scope("weights"):
    W = tf.Variable(tf.zeros([784, 10]))

In [66]:
with tf.name_scope("biases"):
    b = tf.Variable(tf.zeros([10]))

In [67]:
with tf.name_scope("softmax"):
    # y is our prediction
    y = tf.nn.softmax(tf.matmul(x,W) + b)

In [68]:
with tf.name_scope('cross_entropy'):
    # this is our cost
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))


In [69]:
# specify optimizer
with tf.name_scope('train'):
    # optimizer is an "operation" which we can execute in a session
    train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)


In [70]:
with tf.name_scope('Accuracy'):
    # Accuracy
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
   

In [71]:
tf.summary.scalar("cost", cross_entropy)
tf.summary.scalar("accuracy", accuracy)

<tf.Tensor 'accuracy:0' shape=() dtype=string>

In [72]:
summary_op = tf.summary.merge_all()

In [73]:
with tf.Session() as sess:
    # variables need to be initialized before we can use them
    sess.run(tf.initialize_all_variables())

    # create log writer object
    writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
        
    # perform training cycles
    for epoch in range(training_epochs):
        
        # number of batches in one epoch
        batch_count = int(mnist.train.num_examples/batch_size)
        
        for i in range(batch_count):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            
            # perform the operations we defined earlier on batch
            _, summary = sess.run([train_op, summary_op], feed_dict={x: batch_x, y_: batch_y})
            
            # write log
            writer.add_summary(summary, epoch * batch_count + i)
            
        if epoch % 5 == 0: 
            print ("Epoch: ", epoch )
    print ("Accuracy: ", accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))
    print ("done")

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch:  0
Accuracy:  0.9205
done


In [ ]:
#tensorboard --logdir=run1:/tmp/mnist/1,run2:/tmp/mnist/2 --port=6007